In [1]:
import json
import pandas as pd 
import numpy as np 
import re
import urllib.request

In [3]:
def get_hdr():
    return {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [5]:
def requesting(url,headers = None):
    req = urllib.request.Request(url,headers=headers)
    response= urllib.request.urlopen(req)
    data = response.read()
    encoding = response.info().get_content_charset('utf-8')
    data = json.loads(data.decode(encoding))
    return data 

In [16]:
def params_setting(category=None):
    if limit != None: assert(type(limit)==int)
    mode = 'default'
    if category != None:
        if category in ['food','health-well-being']:
            category = 'category='+str(categories[category])
            mode = 'category'
        else:
            print('wrong value for category: will not be considered')
    else:
        category=''
    return mode, category

In [31]:

def get_creators(limit=None,category=None, headers = None):
    '''
       returns creator in a list 
    
       Parameters
       ----------
           limit : the number of creators to get, if not specified return all authors.
                Authors are crawled in the order giben by the site 
                N.B. if not specified can take some time 
           
           category : specifying a category will obtain only authors of that category 

    ''' 
    
    # PARAMETHERS SETTING
    mode ,category = params_setting(category)
    page='1'
    base_url = 'https://api.tipeee.com/v2.0/projects?mode={}&page={}&perPage=150&lang=en{}'
    creators_list = list()
    #COLLECTING DATA
    while len(creators_list) < limit:
        data = requesting(base_url.format(mode, page, category), headers) 
        creators_list += data['items']
        try: 
            page = data['pager']['next']
        except: 
            break
    if len(creators_list) >  limit : creators_list = creators_list[:limit]
        
    return creators_list
    
            
    
        
  
    
    
    
    

In [ ]:
def to_dataframe(creators_list):
    df = pd.DataFrame(columns=['username','categories','language','tipperAmount','tipperNumber','newsNumber'])
    for creator in creators_list :
        

In [ ]:
categories = {'art-culture':'31', 'bd-illustration':'33', 'movies':'34', 'music':'42'}

In [32]:
creators = get_creators(1000,headers = get_hdr())


In [36]:
df = pd.DataFrame(creators)

In [37]:
df

,translations,id,slug,lang,status,tags,categories,avatar,name,parameters,cover
0,"{'en': {'name': 'Naomi Wu- Shenzhen Maker, Tec...",162339,naomiwu,en,OPEN,[],[{'translations': {'de': {'name': 'Geek & Nerd...,"{'id': 1370291, 'filename': '0be29d87b418c69df...","Naomi Wu- Shenzhen Maker, Tech and DIY Enthusi...","{'hidedAmount': '1', 'tipperAmount': 0, 'tippe...",NaN
1,{'en': {'name': 'AndreaVicari'}},242071,andreavicari,en,OPEN,"[{}, {}, {}, {}, {}]","[{'translations': {'de': {'name': 'Musik'}, 'e...","{'id': 1818415, 'filename': '202004075e8cb638b...",,"{'hidedAmount': '0', 'tipperAmount': '88', 'ti...","{'id': 1818414, 'filename': '202004085e8dae814..."
2,{'en': {'name': 'ladyteruki'}},177029,ladyteruki,en,OPEN,"[{}, {}, {}]",[{'translations': {'de': {'name': 'Kunst & Kul...,"{'id': 1494326, 'filename': '202101015fee7bd73...",ladyteruki,"{'hidedAmount': '0', 'tipperAmount': '365', 't...","{'id': 1760962, 'filename': '201911085dc5ac15b..."
3,{'en': {'name': 'Moewxa'}},156945,moewxa,en,OPEN,[],[{'translations': {'de': {'name': 'Kunst & Kul...,"{'id': 1329551, 'filename': '202010015f750a97c...",Moewxa,"{'hidedAmount': '1', 'tipperAmount': 0, 'tippe...","{'id': 1760538, 'filename': '202010015f750a6e6..."
4,{'en': {'name': 'La Renaissance du Plus Puissa...,235051,la-renaissance-du-plus-puissant-des-dieux-epei...,en,OPEN,[],[{'translations': {'de': {'name': 'Kunst & Kul...,"{'id': 1786896, 'filename': '202001175e212deb7...",,"{'hidedAmount': False, 'tipperAmount': '93', '...","{'id': 1786895, 'filename': '202001175e213239c..."
...,...,...,...,...,...,...,...,...,...,...,...
995,"{'fr': {'name': 'Aurélien Fernandez, illustrat...",108213,aurelien-fernandez-illustration-et-bande-dessinee,fr,OPEN,[],[{'translations': {'de': {'name': 'Comics & Ab...,"{'id': 999033, 'filename': '202003235e78dbeee8...",Aurélien Fernandez,"{'hidedAmount': False, 'tipperAmount': '50', '...","{'id': 1809864, 'filename': '202003235e78dc08c..."
996,{'fr': {'name': 'Outside the box (Anciennement...,108751,le-top-5,fr,OPEN,[],[{'translations': {'de': {'name': 'Kunst & Kul...,"{'id': 1002174, 'filename': 'a77c4fd5080326f76...",Le top 5,"{'hidedAmount': False, 'tipperAmount': '42', '...",NaN
997,{},110132,d-mystif,fr,OPEN,[],[{'translations': {'de': {'name': 'Kunst & Kul...,"{'id': 1013162, 'filename': 'b61b872633aa2bcf3...",D-Mystif,"{'hidedAmount': False, 'tipperAmount': '32', '...",NaN
998,{'fr': {'name': 'Otekaï'}},114072,otekai,fr,OPEN,[],[{'translations': {'de': {'name': 'Kunst & Kul...,"{'id': 1045866, 'filename': '202012195fde2540a...",jeromemorel,"{'hidedAmount': '0', 'tipperAmount': '58', 'ti...","{'id': 1800411, 'filename': '20210214602860956..."


In [27]:
assert(type(None)==int)

AssertionError: 